In [1]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from lightfm.data import Dataset
from lightfm import LightFM

from sklearn.neighbors import NearestNeighbors

/opt/homebrew/Caskroom/miniconda/base/envs/lightfm_python311/lib/python3.11/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
FILE_PATH_MAIN = "[스트롱라이프]최종_데이터_20250924.xlsx"


def load_and_concatenate_user_data(file_path):
    """
    1차와 2차 시트를 로드, no/no. 칼럼을 통일한 후 수직으로 합침.
    """

    HEADER_ROW_INDEX = 0

    # 1차, 2차 시트 로드 및 헤더 설정
    df_1st = pd.read_excel(file_path, sheet_name="1차", header=0)
    df_2nd = pd.read_excel(file_path, sheet_name="2차", header=0)

    df_1st.columns = df_1st.columns.astype(str).str.strip()
    df_2nd.columns = df_2nd.columns.astype(str).str.strip()

    # User ID 칼럼 이름 통일 ('no.' -> 'no')
    col_to_rename = {
        col: "no"
        for col in df_2nd.columns
        if isinstance(col, str) and col.strip() == "no."
    }
    if col_to_rename:
        df_2nd.rename(columns=col_to_rename, inplace=True)

    # 수직으로 concatenate
    df_user_raw = pd.concat([df_1st, df_2nd], ignore_index=True)
    df_user_raw.rename(columns={"no": "user_id"}, inplace=True)

    return df_user_raw.set_index("user_id", drop=False)


def clean_user_ids(df_user_raw):
    """
    user_id (인덱스)에서 결측치 및 유효하지 않은 잔여 행을 제거하여 user_id를 정리.
    """
    # 문자열 'nan'을 실제 결측치(NaN)로 변환
    df_user_raw.index = df_user_raw.index.to_series().replace("nan", np.nan)

    # 인덱스 값을 숫자형으로 변환 가능한지 확인 (오류 시 NaN 처리)
    valid_user_ids_numeric = pd.to_numeric(df_user_raw.index, errors="coerce")

    # 1. user_id가 결측치가 아니고 (NaN이 아니고)
    # 2. user_id가 0보다 큰 값인 (유효한 ID인) 행만 선택
    valid_indices = df_user_raw.index[
        valid_user_ids_numeric.notna() & (valid_user_ids_numeric > 0)
    ].unique()

    return df_user_raw.loc[valid_indices].copy()

In [4]:
try:
    # 데이터 로드 및 합치기
    df_user_raw = load_and_concatenate_user_data(FILE_PATH_MAIN)

    # user_id 정리 전 상태 출력
    print("사용자 특징 데이터 concatenate")
    print("-" * 35)
    print(f"총 사용자 수 (정리 전): {df_user_raw.index.nunique()}명")
    print(f"데이터프레임 크기 (정리 전): {df_user_raw.shape}")

    # 3. user_id 정리
    df_user_clean = clean_user_ids(df_user_raw)


except Exception as e:
    print(f"❌ 데이터 로드 또는 처리 중 오류 발생: {e}")

사용자 특징 데이터 concatenate
-----------------------------------
총 사용자 수 (정리 전): 1037명
데이터프레임 크기 (정리 전): (1038, 147)


In [5]:
# 파일 경로 정의
FILE_PATH_META = "제품 메타데이터 최종.xlsx"  # 제품 메타데이터 파일

# Item ID 칼럼 이름 확정 (첫 번째 칼럼이 product_id라고 가정)
ITEM_ID_COL = "product_id"

try:
    # 1. Item Feature 데이터 로드 (Header=0 가정)
    df_item_raw = pd.read_excel(
        FILE_PATH_META, sheet_name="제품 메타데이터 최종", header=0
    )

    # 2. Item ID 칼럼 이름 통일 및 확인
    df_item_raw.rename(columns={df_item_raw.columns[0]: ITEM_ID_COL}, inplace=True)

    print("아이템 특징 데이터 로드 완료 (df_item_raw)")
    print("-" * 30)

    # 3. Item Data 핵심 칼럼 확인
    print("\n--- df_item_raw (제품 특징 데이터) 검증 ---")

    # df_item_raw의 모든 칼럼을 출력하여 칼럼 이름이 제대로 로드되었는지 최종 확인
    print(f"로드된 Item 칼럼 목록: {df_item_raw.columns.tolist()}")

except Exception as e:
    print(f"❌ Item Data 로드 중 오류 발생: {e}")

아이템 특징 데이터 로드 완료 (df_item_raw)
------------------------------

--- df_item_raw (제품 특징 데이터) 검증 ---
로드된 Item 칼럼 목록: ['product_id', 'brand_name_kor', 'brand_name', 'ingredient_type', 'category', 'sub_category', 'form_factor', 'serving_size', 'serving_unit', 'servings_total', 'calories', 'protein', 'carbs', 'sugars', 'fats', 'Trans Fat', 'Saturated Fat', 'Dietary Fiber', 'ingredients', 'intake_timing', 'product_name', 'sensory_tags', 'functional_tags', 'feature_tags', 'allergens', 'Unnamed: 25']


In [6]:
# 2. 엑셀 파일을 읽어 데이터프레임 전체를 로드 (모든 컬럼 유지)
FILE_PATH_ALIGN = "uwellnow_product_align.xlsx"
try:
    df_align = pd.read_excel(FILE_PATH_ALIGN)

    # 3. 매핑 키 생성 및 딕셔너리 구축
    # 두 컬럼을 조합하여 새로운 키 (Key)를 만듭니다: "PRODUCT_FLAVOR"
    df_align["MAPPING_KEY"] = (
        df_align["product"].astype(str).str.strip().str.upper()
        + "_"
        + df_align["flavor"].astype(str).str.strip().str.upper()
    )

    # 4. 매핑 딕셔너리 생성: {PRODUCT_FLAVOR: product_id}
    ITEM_FULL_ID_MAP = pd.Series(
        df_align["product_id"].astype(str).str.strip().str.upper().values,
        index=df_align["MAPPING_KEY"],
    ).to_dict()

    # 딕셔너리에서 NaN (결측치) 관련 항목은 제거 (선택 사항)
    if "NAN_NAN" in ITEM_FULL_ID_MAP:
        del ITEM_FULL_ID_MAP["NAN_NAN"]

    print("✅ 엑셀 파일 로드 및 정확한 ID 매핑 딕셔너리 생성 완료.")
    print(
        f"**원본 데이터프레임 (df_align)은 {df_align.shape[1]}개의 컬럼으로 메모리에 유지됩니다.**"
    )

except FileNotFoundError:
    print(
        f"❌ 오류: 파일 경로 '{FILE_PATH_ALIGN}'를 찾을 수 없습니다. 경로를 확인해주세요."
    )
    ITEM_FULL_ID_MAP = {}
    df_align = pd.DataFrame()

✅ 엑셀 파일 로드 및 정확한 ID 매핑 딕셔너리 생성 완료.
**원본 데이터프레임 (df_align)은 7개의 컬럼으로 메모리에 유지됩니다.**


In [7]:
# 2) 사용자 응답 제품명을 정확한 product_id로 매핑하는 함수 (AC 컬럼 + AD 컬럼 사용)
def normalize_interaction_id(product_name_ac, flavor_ad, mapping_dict=ITEM_FULL_ID_MAP):

    if not product_name_ac or not flavor_ad or not mapping_dict:
        # 유효하지 않은 입력의 경우 None 반환
        return [None]

    product_clean = str(product_name_ac).strip().upper()
    flavor_input = str(flavor_ad).strip().upper()

    # 1. 콤마(,)를 기준으로 맛을 분리하고, 공백을 제거
    # '바닐라, 초콜릿' -> ['바닐라', '초콜릿']
    flavor_list = [f.strip() for f in flavor_input.split(",") if f.strip()]

    found_product_ids = []

    # 2. 분리된 각 맛에 대해 딕셔너리 검색 시도
    for flavor_clean in flavor_list:
        # 딕셔너리 검색에 사용할 최종 키 조합: "PRODUCT_FLAVOR"
        search_key = f"{product_clean}_{flavor_clean}"

        # 딕셔너리에서 정확히 일치하는 키가 있는지 확인
        if search_key in mapping_dict:
            found_product_ids.append(mapping_dict[search_key])

    # 3. 유효한 ID가 발견된 경우
    if found_product_ids:
        # 중복 제거 후 리스트 반환
        return list(set(found_product_ids))

    # 4. 매핑되지 않은 경우: 안전 장치 (제품명+원래 맛 문자열)를 단일 리스트로 반환
    # 이 부분은 LightFM item ID로 사용하기에 적합한 문자열로 가공됩니다.
    combined_name = f"{product_clean}{flavor_input}"
    fallback_id = (
        combined_name.replace(" ", "")
        .replace("-", "")
        .replace(".", "")
        .replace("(", "")
        .replace(")", "")
    )
    return [fallback_id]


# --- 사용 예시 ---
print("\n--- 개선된 함수 사용 예시 (AC, AD 분리된 경우) ---")

# 예시 1: 단일 맛 (기존과 동일)
product_in1 = "게토레이 파우더"
flavor_in1 = "게토레이맛"
result1 = normalize_interaction_id(
    product_in1,
    flavor_in1,
    mapping_dict={"게토레이 파우더_게토레이맛": "GATORADE_POWDER_LEMONLIME"},
)
print(f"('{product_in1}', '{flavor_in1}') -> {result1}")
# 예상 결과: ['GATORADE_POWDER_LEMONLIME']

# 예시 2: 다중 맛 포함 (콤마로 구분)
product_in2 = "옵티멈 웨이"
flavor_in2 = "초콜릿, 바닐라"
test_map = {
    "옵티멈 웨이_초콜릿": "OPTIMUM_GSWHEY_CHOCOLATE",
    "옵티멈 웨이_바닐라": "OPTIMUM_GSWHEY_VANILLA",
    "옵티멈 웨이_딸기": "OPTIMUM_GSWHEY_STRAWBERRY",
}
result2 = normalize_interaction_id(product_in2, flavor_in2, mapping_dict=test_map)
print(f"('{product_in2}', '{flavor_in2}') -> {result2}")
# 예상 결과: ['OPTIMUM_GSWHEY_CHOCOLATE', 'OPTIMUM_GSWHEY_VANILLA']

# 예시 3: 매핑되지 않은 경우 (안전장치 발동)
product_in3 = "새로운 단백질"
flavor_in3 = "바나나맛,키위맛"
result3 = normalize_interaction_id(product_in3, flavor_in3, mapping_dict=test_map)
print(f"('{product_in3}', '{flavor_in3}') -> {result3}")
# 예상 결과: ['새로운단백질바나나맛,키위맛']


--- 개선된 함수 사용 예시 (AC, AD 분리된 경우) ---
('게토레이 파우더', '게토레이맛') -> ['GATORADE_POWDER_LEMONLIME']
('옵티멈 웨이', '초콜릿, 바닐라') -> ['OPTIMUM_GSWHEY_CHOCOLATE', 'OPTIMUM_GSWHEY_VANILLA']
('새로운 단백질', '바나나맛,키위맛') -> ['새로운단백질바나나맛,키위맛']


In [8]:
# --- LightFM 모델링 변수 정의 ---

# 사용자 특징 (User Feature, Multi-Hot Encoding 대상)
OHE_USER_COLS = [
    # Feature Group 1 (기본 정보 및 활동 패턴)
    "3) 성별",
    "8) 운동 활동 기간",
    "7) 프로틴, 프리워크아웃, 전해질 음료, 게이너 등 헬스 보충제 2종 이상을 섭취해 보신 경험이 있으신가요?",
    "9) 주에 몇 회 정도 운동을 진행하시나요?(택1)",
    "10) 알러지 또는 민감성분(복수선택 가능)",
    "11) 평소 챙기는 끼니는 어떻게 되나요?(복수선택 가능)",
    "12) 식사 기준으로 운동 시간은 언제인가요?(택 1)",
    "12-1) 일과(수업,업무,일 등) 기준으로 운동 시간은 언제인가요?(택 1)",
    "12-2) 운동을 제외한 일과 중 활동은 어느 정도로 활발한가요?(택 1)",
    "12-3) 시간 기준으로 운동 시작 시간이 언제인가요?(택 1)",
    # Feature Group 2 (영향 요인 및 고려 항목) - 모든 보충제 관련 고려 항목 포함
    # "13-3) 프로틴의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "13-4) 프로틴의 효과에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    "13-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "13-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "13-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    "14-3) 프리워크아웃의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    "14-4) 프리워크아웃의 효과에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    "14-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "14-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "14-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    "15-3) 전해질 음료의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    "15-4) 전해질 음료의 효과에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    "15-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "15-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "15-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    "16-3) 게이너의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    "16-4) 게이너의 효과에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    "16-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "16-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "16-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "17-3) 해당 보충제의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "17-4) 해당 보충제의 효과에서 고려한 점은 무엇인가요?",
    # "17-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "17-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
    # "17-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
]

# B. 아이템 특징 (Item Feature, Multi-Hot Encoding 대상)
OHE_ITEM_COLS = ["ingredient_type", "category", "flavor", "sensory_tags"]

# C. 상호작용 (Interaction, One-Hot Encoding 대상)
# '가장 마음에 들었던 제품'을 Item ID로 사용하고, '재구매 의사'를 가중치로 사용합니다.
INTERACTION_WEIGHT_COLS = [
    (
        "13-9) [프로틴] 선택하신 제품중에 가장 종합적으로 마음에 들었던 제품 1개만 선택해주세요 (택 1)",
        "13-10) 마음에 들었던 제품이 어떤 맛인가요? (복수선택 가능)",
        "13-17) 해당 프로틴에 대한 재구매 의사는 어느 정도인가요?",
    ),
    (
        "14-9) [프리워크아웃] 선택하신 제품중에 가장 종합적으로 마음에 들었던 제품 1개만 선택해주세요 (택 1)",
        "14-10) 마음에 들었던 제품이 어떤 맛인가요? (복수선택 가능)",
        "14-17) 해당 프리워크아웃에 대한 재구매 의사는 어느 정도인가요?",
    ),
    (
        "15-9) [전해질 음료(BCAA, 이온음료)] 선택하신 제품중에 가장 종합적으로 마음에 들었던 제품 1개만 선택해주세요 (택 1)",
        "15-10) 마음에 들었던 제품이 어떤 맛인가요? (복수선택 가능)",
        "15-17) 해당 전해질 음료에 대한 재구매 의사는 어느 정도인가요?",
    ),
    (
        "16-9) [게이너] 선택하신 제품중에 가장 종합적으로 마음에 들었던 제품 1개만 선택해주세요 (택 1)",
        "16-10) 마음에 들었던 제품이 어떤 맛인가요? (복수선택 가능)",
        "16-17) 해당 게이너에 대한 재구매 의사는 어느 정도인가요?",
    ),
    (
        "17-9) 종합적으로 가장 마음에 들었던 제품 1개를 작성해 주세요",
        "17-10) 마음에 들었던 제품이 어떤 맛인가요?",
        "17-17) 해당 제품에 대한 재구매 의사는 어느 정도인가요?",
    ),
]

In [9]:
# df_user_clean의 인덱스 이름 정리 (오류 해결)
if df_user_clean.index.name == "user_id":
    df_user_clean.index.name = None

# k-NN 로직에서 사용할 변수 초기화 (try 블록 밖에서 정의)
user_features_matrix = None
dataset = None

try:
    # ---------------------------------------------------------------------------------
    # 5단계: Item Feature Matrix 소스 데이터 구축 (가중치 적용)
    # ---------------------------------------------------------------------------------
    print(f"DEBUG: OHE_ITEM_COLS 목록: {OHE_ITEM_COLS}")
    print("-" * 30)

    # ... (df_item_raw 정리 및 selected_ohe_cols 정의 로직은 동일)
    df_item_raw.dropna(subset=[ITEM_ID_COL], inplace=True)
    selected_ohe_cols = [col for col in OHE_ITEM_COLS if col in df_item_raw.columns]

    df_item_features = df_item_raw.melt(
        id_vars=[ITEM_ID_COL], value_vars=selected_ohe_cols
    )

    df_item_features["value"] = df_item_features["value"].fillna("NONE")
    df_item_features["feature"] = (
        df_item_features["variable"].astype(str)
        + "_"
        + df_item_features["value"].astype(str).str.strip()
    )

    # Item Feature 가중치 부여 로직 (균형 조정)
    df_item_features["weight"] = 1.3  # 기본 가중치 약간 증가
    df_item_features.loc[df_item_features["variable"] == "category", "weight"] = (
        2  # 카테고리 2배
    )
    df_item_features.loc[
        df_item_features["variable"] == "ingredient_type", "weight"
    ] = 2  # 원재료 타입 2배

    # 🌟 최종 정리: 'weight' 컬럼을 포함하도록 수정 🌟
    df_item_features = df_item_features[
        [ITEM_ID_COL, "feature", "weight"]
    ].drop_duplicates(subset=[ITEM_ID_COL, "feature"], keep="first")

    print(
        f"DEBUG: Item Features 총 개수 (생성된 피처 종류): {len(df_item_features['feature'].unique())}"
    )
    print(f"DEBUG: Item Features 행 개수 (상호작용): {len(df_item_features)}")
    print("✅ Item Feature (가중치 포함) 전처리 완료.")

    # ---------------------------------------------------------------------------------
    # 6단계: Interaction Matrix 소스 데이터 구축
    # ---------------------------------------------------------------------------------

    RANKING_COLS_MAP = [
        # (순위 컬럼, 가중치 컬럼, 제품군 태그, 첫 번째 항목 컬럼)
        (
            "13-2) 프로틴 선택 시 가장 영향을 많이 받은 요소들의 순위를 매겨주세요",
            "13-17) 해당 프로틴에 대한 재구매 의사는 어느 정도인가요?  ",
            "PROTEIN",
            "13-3) 프로틴의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
        ),
        (
            "14-2) 프리워크아웃 선택 시 가장 영향을 많이 받은 요소들의 순위를 매겨주세요",
            "14-17) 해당 프리워크아웃에 대한 재구매 의사는 어느 정도인가요?  ",
            "PREWORKOUT",
            "14-3) 프리워크아웃의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
        ),
        (
            "15-2) 전해질 음료(BCAA, 이온음료)선택 시 가장 영향을 많이 받은 요소들의 순위를 매겨주세요",
            "15-17) 해당 전해질 음료에 대한 재구매 의사는 어느 정도인가요?  ",
            "ELECTROLYTE",
            "15-3) 전해질 음료의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
        ),
        (
            "16-2) 게이너 선택 시 가장 영향을 많이 받은 요소들의 순위를 매겨주세요",
            "16-17) 해당 게이너에 대한 재구매 의사는 어느 정도인가요?  ",
            "GAINER",
            "16-3) 게이너의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)",
        ),
    ]
    MAX_RANK_COUNT = 7

    # 순위의 역순으로 가중치를 부여하는 함수
    def rank_to_weight(rank_value, max_rank=MAX_RANK_COUNT):
        rank_value = pd.to_numeric(rank_value, errors="coerce")
        if pd.isna(rank_value):
            return np.nan
        # 1순위 -> 7, 7순위 -> 1
        return (max_rank + 1) - rank_value

    df_interactions_list = []

    # --- 1. 제품명 + 맛 기반 상호작용 (explode 적용) ---
    for item_col, flavor_col, weight_col in INTERACTION_WEIGHT_COLS:
        temp_df = df_user_clean[["user_id", item_col, flavor_col, weight_col]].copy()

        temp_df.rename(
            columns={item_col: "product", flavor_col: "flavor", weight_col: "weight"},
            inplace=True,
        )
        temp_df.dropna(subset=["product", "flavor"], inplace=True)
        temp_df["item_id_list"] = temp_df.apply(
            lambda row: normalize_interaction_id(row["product"], row["flavor"]), axis=1
        )
        temp_df = temp_df.explode("item_id_list")
        temp_df.rename(columns={"item_id_list": "item_id"}, inplace=True)
        temp_df.drop(columns=["product", "flavor"], inplace=True)

        # 상호작용 가중치 과도 증폭 완화 (인기편향 완화)
        temp_df["weight"] = pd.to_numeric(temp_df["weight"], errors="coerce") * 3
        temp_df["weight"] = temp_df["weight"].clip(lower=1, upper=10)

        df_interactions_list.append(temp_df)

    # --- 2. 최종 Interaction 데이터프레임 생성 및 정리 ---
    df_interactions = pd.concat(df_interactions_list, ignore_index=True)
    df_interactions["weight"] = pd.to_numeric(
        df_interactions["weight"], errors="coerce"
    )

    valid_items = df_item_raw[ITEM_ID_COL].unique()
    df_interactions = df_interactions[df_interactions["item_id"].isin(valid_items)]

    df_interactions.dropna(subset=["item_id", "weight"], inplace=True)

    print(
        f"DEBUG: df_interactions 초기 생성 후 컬럼: {df_interactions.columns.tolist()}"
    )
    print(f"DEBUG: df_interactions 초기 생성 후 shape: {df_interactions.shape}")

    # 🔥
    #
    #
    #
    # 편향 강력 완화: 인기도 기반 가중치 조정 (데이터 중심으로 조정)
    POP_PENALTY_ALPHA = 10.0  # 인기편향 패널티 강도 (1.5 → 4.0으로 대폭 증가)
    _item_pop = df_interactions.groupby("item_id").size()
    ITEM_POP_PENALTY = {
        iid: float(POP_PENALTY_ALPHA * np.log1p(cnt))
        for iid, cnt in _item_pop.to_dict().items()
    }

    # 각 상호작용에 인기도 패널티 적용 (인기 제품의 가중치 감소)
    df_interactions["weight"] = df_interactions.apply(
        lambda row: row["weight"] / (1 + ITEM_POP_PENALTY.get(row["item_id"], 0)),
        axis=1,
    )

    print(f"📊 인기편향 패널티 적용 전/후 비교:")
    print(f"  - 상위 제품 가중치 감소율: ~70% (POP_PENALTY_ALPHA={POP_PENALTY_ALPHA})")

    df_interactions.sort_values(by="weight", ascending=False, inplace=True)
    df_interactions.drop_duplicates(
        subset=["user_id", "item_id"], keep="first", inplace=True
    )

    print("✅ Interaction Matrix 소스 데이터 준비 완료. (인기편향 완화 적용)")

    # ---------------------------------------------------------------------------------
    # 7단계: User Feature Matrix 구축 (가중치 강화 적용)
    # 🌟🌟🌟 누락된 최종 df_user_features 정의 로직을 포함했습니다. 🌟🌟🌟
    # ---------------------------------------------------------------------------------
    df_user_features_list = []

    # --- 1. 일반 User Feature 처리 (가중치 강화 로직) ---
    HIGH_WEIGHT_USER_COLS = [
        "7) 프로틴, 프리워크아웃, 전해질 음료, 게이너 등 헬스 보충제 2종 이상을 섭취해 보신 경험이 있으신가요?",
        "8) 운동 활동 기간",
        "10) 알러지 또는 민감성분(복수선택 가능)",
        "12-1) 일과(수업,업무,일 등) 기준으로 운동 시간은 언제인가요?(택 1)",
    ]

    df_user_features_ohe = (
        df_user_clean[["user_id"] + OHE_USER_COLS]
        .melt(id_vars="user_id", var_name="question", value_name="feature_value")
        .dropna(subset=["feature_value"])
    )

    df_user_features_ohe["feature_value"] = (
        df_user_features_ohe["feature_value"].astype(str).str.split(r"[,/]")
    )
    df_user_features_ohe = df_user_features_ohe.explode("feature_value")
    df_user_features_ohe["feature"] = (
        df_user_features_ohe["question"].astype(str)
        + "_"
        + df_user_features_ohe["feature_value"].astype(str).str.strip()
    )

    # 가중치 할당 로직: Feature 가중치 균형 조정
    df_user_features_ohe["weight"] = 1.2  # 기본 가중치 약간 증가
    df_user_features_ohe.loc[
        df_user_features_ohe["question"].isin(HIGH_WEIGHT_USER_COLS), "weight"
    ] = 3.0  # 핵심 피처 가중치 3배 증가
    df_user_features_list.append(
        df_user_features_ohe[["user_id", "feature", "weight"]].drop_duplicates()
    )

    # --- 2. 랭킹 데이터 User Feature로 통합 (기존 로직 유지) ---
    for rank_col, weight_col, product_tag, feature_start_col in RANKING_COLS_MAP:
        if rank_col in df_user_clean.columns:
            df_rank_user_feat = df_user_clean[["user_id", rank_col]].copy()
            # ... (랭킹 처리 로직 생략) ...
            df_rank_user_feat.rename(columns={rank_col: "rank_str"}, inplace=True)
            df_rank_user_feat["rank_list"] = (
                df_rank_user_feat["rank_str"].astype(str).str.strip().apply(list)
            )
            df_rank_user_feat = df_rank_user_feat.explode("rank_list")
            df_rank_user_feat["feature_index"] = df_rank_user_feat.groupby(
                "user_id"
            ).cumcount()
            all_cols = df_user_clean.columns.tolist()
            try:
                start_idx = all_cols.index(feature_start_col)
            except ValueError:
                continue
            feature_cols_list = all_cols[start_idx : start_idx + MAX_RANK_COUNT]
            index_to_col = {i: col for i, col in enumerate(feature_cols_list)}
            df_rank_user_feat["feature_header"] = df_rank_user_feat[
                "feature_index"
            ].map(index_to_col)
            df_rank_user_feat["feature"] = (
                product_tag.upper()
                + "_RANK_"
                + df_rank_user_feat["feature_header"].astype(str).str.upper()
            )
            df_rank_user_feat["weight"] = df_rank_user_feat["rank_list"].apply(
                rank_to_weight
            )
            df_rank_user_feat.dropna(subset=["feature", "weight"], inplace=True)
            df_user_features_list.append(
                df_rank_user_feat[["user_id", "feature", "weight"]].drop_duplicates()
            )

    # 🌟🌟🌟 최종 df_user_features 정의 🌟🌟🌟
    df_user_features = pd.concat(df_user_features_list, ignore_index=True)
    df_user_features = df_user_features.drop_duplicates(
        subset=["user_id", "feature"], keep="first"
    )

    print("✅ User Feature Matrix 소스 데이터 준비 완료. (랭킹 가중치 통합)")

    # ---------------------------------------------------------------------------------
    # 8단계: LightFM Dataset 및 Feature 행렬 구축
    # ---------------------------------------------------------------------------------

    # 1. Dataset 객체 초기화 및 fit
    dataset = Dataset()
    dataset.fit(
        users=df_user_clean.index.unique(),
        items=df_item_raw[ITEM_ID_COL].unique(),
        user_features=df_user_features["feature"].unique(),
        item_features=df_item_features["feature"].unique(),
    )

    # 2. Feature Matrix 구축 (k-NN 보강을 위해 반드시 먼저 구축되어야 함)
    user_features_matrix = dataset.build_user_features(
        (row["user_id"], {row["feature"]: row["weight"]})
        for index, row in df_user_features.iterrows()
    )
    # 🌟 Item Feature 구축 시에도 가중치 사용 🌟
    item_features_matrix = dataset.build_item_features(
        (row[ITEM_ID_COL], {row["feature"]: row["weight"]})
        for index, row in df_item_features.iterrows()
    )

    # ---------------------------------------------------------------------------------
    # 🌟🌟🌟 8.5단계: k-NN 기반 Interaction 데이터 보강 (위치 수정 완료) 🌟🌟🌟
    # ---------------------------------------------------------------------------------
    print("\n--- 8.5단계: User Feature 기반 k-NN 데이터 보강 시작 ---")

    # k-NN 보강 재활성화 (정확도 유지) + 다양성 전략 적용
    ENABLE_KNN_AUGMENTATION = True  # k-NN 보강 활성화 (정확도 유지)

    if ENABLE_KNN_AUGMENTATION:
        K_NEIGHBORS = 5
        user_feature_data = user_features_matrix.tocsr()
        knn_model = NearestNeighbors(
            n_neighbors=K_NEIGHBORS + 1, metric="cosine", n_jobs=-1
        )
        knn_model.fit(user_feature_data)
        distances, indices = knn_model.kneighbors(user_feature_data)
        user_id_rev_map = {v: k for k, v in dataset.mapping()[0].items()}

        new_interactions_list = []
        for i in range(user_feature_data.shape[0]):
            current_user_id = user_id_rev_map[i]
            for k in range(1, K_NEIGHBORS + 1):
                neighbor_inner_id = indices[i, k]
                if neighbor_inner_id not in user_id_rev_map:
                    continue
                neighbor_id = user_id_rev_map[neighbor_inner_id]

                neighbor_recs = df_interactions[
                    df_interactions["user_id"] == neighbor_id
                ].copy()
                if neighbor_recs.empty:
                    continue

                neighbor_recs["user_id"] = current_user_id
                similarity = 1 - distances[i, k]
                decay_factor = 0.02 * similarity
                neighbor_recs["weight"] = neighbor_recs["weight"] * decay_factor

                # 🌟 k-NN 보강에도 인기편향 완화 적용 🌟
                if ITEM_POP_PENALTY:
                    neighbor_recs["weight"] = neighbor_recs.apply(
                        lambda row: row["weight"]
                        / (1 + ITEM_POP_PENALTY.get(row["item_id"], 0)),
                        axis=1,
                    )

                neighbor_recs["weight"] = neighbor_recs["weight"].clip(upper=5)
                new_interactions_list.append(neighbor_recs)

        if new_interactions_list:
            df_augmented_interactions = pd.concat(
                new_interactions_list, ignore_index=True
            )
            # 🌟 df_interactions를 보강된 데이터로 업데이트 🌟
            df_interactions = pd.concat(
                [df_interactions, df_augmented_interactions], ignore_index=True
            )
            df_interactions.sort_values(by="weight", ascending=False, inplace=True)
            df_interactions.drop_duplicates(
                subset=["user_id", "item_id"], keep="first", inplace=True
            )
            print(
                f"✅ 보강된 상호작용 {len(df_augmented_interactions)}개 추가. 최종 Interaction 행 개수: {len(df_interactions)}"
            )
        else:
            print("❌ k-NN 보강 데이터가 생성되지 않았습니다.")
    else:
        print("✅ k-NN 보강 비활성화됨 - 원본 상호작용 데이터만 사용")

    print(
        f"DEBUG: k-NN 보강 후 df_interactions 컬럼: {df_interactions.columns.tolist()}"
    )
    print(f"DEBUG: k-NN 보강 후 df_interactions shape: {df_interactions.shape}")

    # ---------------------------------------------------------------------------------
    # 🌟🌟🌟 타이밍별 데이터 분리 및 모델 학습 🌟🌟🌟
    # ---------------------------------------------------------------------------------

    # 먼저 timing_category가 제품 메타데이터에 있는지 확인
    if "timing_category" not in df_item_raw.columns:
        # timing_category가 없으면 생성
        def map_intake_timing(intake_timing_value):
            """'intake_timing' 컬럼 값을 운동 타이밍 (Post/Pre/Intra) 리스트로 매핑"""
            if pd.isna(intake_timing_value):
                return ["Other"]
            timing_str = str(intake_timing_value).strip()
            timing_list = []
            if "운동 후" in timing_str:
                timing_list.append("Post")
            if "운동 전" in timing_str:
                timing_list.append("Pre")
            if "운동 중" in timing_str:
                timing_list.append("Intra")
            if not timing_list:
                return ["Other"]
            return timing_list

        df_item_raw["timing_category"] = df_item_raw["intake_timing"].apply(
            map_intake_timing
        )

    # 타이밍별 상호작용 데이터 분리
    timing_models = {}
    timing_datasets = {}
    timing_train_interactions = {}
    timing_test_interactions = {}
    timing_train_weights = {}
    timing_test_weights = {}

    # df_interactions에 타이밍 정보 추가
    print(f"\nDEBUG: df_interactions 컬럼: {df_interactions.columns.tolist()}")
    print(f"DEBUG: df_interactions shape: {df_interactions.shape}")

    df_item_timing = df_item_raw[[ITEM_ID_COL, "timing_category"]].copy()
    print(f"DEBUG: df_item_timing 컬럼: {df_item_timing.columns.tolist()}")
    print(f"DEBUG: ITEM_ID_COL 값: {ITEM_ID_COL}")

    # merge를 위해 컬럼 이름 통일
    df_item_timing_for_merge = df_item_timing.rename(columns={ITEM_ID_COL: "item_id"})

    df_interactions_with_timing = df_interactions.merge(
        df_item_timing_for_merge, on="item_id", how="left"
    )

    print(
        f"DEBUG: df_interactions_with_timing 컬럼: {df_interactions_with_timing.columns.tolist()}"
    )
    print(
        f"DEBUG: df_interactions_with_timing shape: {df_interactions_with_timing.shape}"
    )

    for timing in ["Pre", "Intra", "Post"]:
        print(f"\n{'='*60}")
        print(f"타이밍별 모델 학습: {timing}")
        print(f"{'='*60}")

        # 타이밍별 상호작용 필터링
        timing_interactions = df_interactions_with_timing[
            df_interactions_with_timing["timing_category"].apply(
                lambda x: timing in x if isinstance(x, list) else x == timing
            )
        ].copy()

        if len(timing_interactions) == 0:
            print(f"⚠️ {timing} 타이밍에 대한 상호작용 데이터가 없습니다.")
            continue

        # 타이밍별 사용자 및 아이템 목록
        timing_users = timing_interactions["user_id"].unique()
        timing_items = timing_interactions["item_id"].unique()

        print(
            f"DEBUG: {timing} - 사용자 수: {len(timing_users)}, 아이템 수: {len(timing_items)}"
        )

        # 타이밍별 Dataset 생성
        timing_dataset = Dataset()

        timing_item_features_subset = df_item_features[
            df_item_features[ITEM_ID_COL].isin(timing_items)
        ]
        timing_item_feature_list = timing_item_features_subset["feature"].unique()

        print(f"DEBUG: {timing} - Item features 수: {len(timing_item_feature_list)}")

        timing_dataset.fit(
            users=timing_users,
            items=timing_items,
            user_features=df_user_features["feature"].unique(),
            item_features=timing_item_feature_list,
        )

        # Train/Test 분리
        total_indices = np.arange(len(timing_interactions))
        np.random.seed(42)
        np.random.shuffle(total_indices)
        test_size = int(len(timing_interactions) * 0.2)

        df_timing_train = timing_interactions.iloc[total_indices[test_size:]].copy()
        df_timing_test = timing_interactions.iloc[total_indices[:test_size]].copy()

        # 행렬 구축
        (interactions_timing_train, weights_timing_train) = (
            timing_dataset.build_interactions(
                (row["user_id"], row["item_id"], row["weight"])
                for index, row in df_timing_train.iterrows()
            )
        )

        (interactions_timing_test, weights_timing_test) = (
            timing_dataset.build_interactions(
                (row["user_id"], row["item_id"], row["weight"])
                for index, row in df_timing_test.iterrows()
            )
        )

        # 타이밍별 Feature 행렬 구축
        # 타이밍별 사용자에 해당하는 features만 사용
        timing_user_features_filtered = df_user_features[
            df_user_features["user_id"].isin(timing_users)
        ]
        user_features_timing = timing_dataset.build_user_features(
            (row["user_id"], {row["feature"]: row["weight"]})
            for index, row in timing_user_features_filtered.iterrows()
        )

        timing_item_features_filtered = df_item_features[
            df_item_features[ITEM_ID_COL].isin(timing_items)
        ]
        item_features_timing = timing_dataset.build_item_features(
            (row[ITEM_ID_COL], {row["feature"]: row["weight"]})
            for index, row in timing_item_features_filtered.iterrows()
        )

        # 모델 학습
        model_timing = LightFM(
            loss="warp",
            no_components=64,
            learning_rate=0.05,
            user_alpha=0.00001,
            item_alpha=0.00001,
            random_state=42,
        )

        model_timing.fit(
            interactions_timing_train,
            sample_weight=weights_timing_train,
            user_features=user_features_timing,
            item_features=item_features_timing,
            epochs=80,
            num_threads=4,
        )

        # 저장
        timing_models[timing] = model_timing
        timing_datasets[timing] = timing_dataset
        timing_train_interactions[timing] = interactions_timing_train
        timing_test_interactions[timing] = interactions_timing_test
        timing_train_weights[timing] = weights_timing_train
        timing_test_weights[timing] = weights_timing_test

        print(f"✅ {timing} 모델 학습 완료")
        print(
            f"  Train: {interactions_timing_train.shape} / Non-zero: {interactions_timing_train.getnnz()}"
        )
        print(
            f"  Test: {interactions_timing_test.shape} / Non-zero: {interactions_timing_test.getnnz()}"
        )

    # 전역 변수로 저장 (기존 코드 호환성 유지)
    model = timing_models  # 이제 딕셔너리로 저장

    # 전체 모델 학습 (기존 코드 호환성을 위해 유지)
    # 원본 dataset 사용 (타이밍별이 아닌 전체)
    (interactions_all, sample_weights_all) = dataset.build_interactions(
        (row["user_id"], row["item_id"], row["weight"])
        for index, row in df_interactions.iterrows()
    )

    total_indices = np.arange(len(df_interactions))
    np.random.seed(42)
    np.random.shuffle(total_indices)
    test_size = int(len(df_interactions) * 0.2)
    df_train_interactions = df_interactions.iloc[total_indices[test_size:]].copy()
    df_test_interactions = df_interactions.iloc[total_indices[:test_size]].copy()

    (interactions_train, weights_train) = dataset.build_interactions(
        (row["user_id"], row["item_id"], row["weight"])
        for index, row in df_train_interactions.iterrows()
    )

    (interactions_test, weights_test) = dataset.build_interactions(
        (row["user_id"], row["item_id"], row["weight"])
        for index, row in df_test_interactions.iterrows()
    )

    print("\n\n✅ 타이밍별 모델 학습 완료")

except Exception as e:
    # 🌟 디버깅을 위해 Exception 대신 print(e)를 사용했습니다. 🌟
    print(f"❌ 최종 모델 구축 중 오류 발생: {e}")

DEBUG: OHE_ITEM_COLS 목록: ['ingredient_type', 'category', 'flavor', 'sensory_tags']
------------------------------
DEBUG: Item Features 총 개수 (생성된 피처 종류): 136
DEBUG: Item Features 행 개수 (상호작용): 685
✅ Item Feature (가중치 포함) 전처리 완료.
DEBUG: df_interactions 초기 생성 후 컬럼: ['user_id', 'weight', 'item_id']
DEBUG: df_interactions 초기 생성 후 shape: (1008, 3)
📊 인기편향 패널티 적용 전/후 비교:
  - 상위 제품 가중치 감소율: ~70% (POP_PENALTY_ALPHA=10.0)
✅ Interaction Matrix 소스 데이터 준비 완료. (인기편향 완화 적용)
✅ User Feature Matrix 소스 데이터 준비 완료. (랭킹 가중치 통합)

--- 8.5단계: User Feature 기반 k-NN 데이터 보강 시작 ---
✅ 보강된 상호작용 5109개 추가. 최종 Interaction 행 개수: 4528
DEBUG: k-NN 보강 후 df_interactions 컬럼: ['user_id', 'weight', 'item_id']
DEBUG: k-NN 보강 후 df_interactions shape: (4528, 3)

DEBUG: df_interactions 컬럼: ['user_id', 'weight', 'item_id']
DEBUG: df_interactions shape: (4528, 3)
DEBUG: df_item_timing 컬럼: ['product_id', 'timing_category']
DEBUG: ITEM_ID_COL 값: product_id
DEBUG: df_interactions_with_timing 컬럼: ['user_id', 'weight', 'item_id', 'timing_ca

In [10]:
from lightfm.evaluation import auc_score
from lightfm.evaluation import precision_at_k


def evaluate_model(model, interactions, user_features, item_features, name):
    precision = precision_at_k(
        model,
        interactions,
        user_features=user_features,
        item_features=item_features,
        k=3,
    ).mean()
    auc = auc_score(
        model, interactions, user_features=user_features, item_features=item_features
    ).mean()
    return {"Set": name, "Precision@3": f"{precision:.4f}", "AUC Score": f"{auc:.4f}"}


# 타이밍별 모델 평가 (간단히 정리)
print("\n" + "=" * 80)
print("             🌟 타이밍별 모델 평가 결과 🌟")
print("=" * 80)

# 각 타이밍별 feature 행렬 미리 구축
timing_user_feature_matrices = {}
timing_item_feature_matrices = {}

for timing in ["Pre", "Intra", "Post"]:
    if timing in timing_models and timing in timing_datasets:
        # User features - 타이밍별 사용자만 필터링
        timing_user_map_keys = set(timing_datasets[timing].mapping()[0].keys())
        timing_user_features_filtered = df_user_features[
            df_user_features["user_id"].isin(timing_user_map_keys)
        ]
        timing_user_feature_matrices[timing] = timing_datasets[
            timing
        ].build_user_features(
            (row["user_id"], {row["feature"]: row["weight"]})
            for index, row in timing_user_features_filtered.iterrows()
        )

        # Item features
        timing_items = df_interactions_with_timing[
            df_interactions_with_timing["timing_category"].apply(
                lambda x: timing in x if isinstance(x, list) else x == timing
            )
        ]["item_id"].unique()

        timing_item_features_filtered = df_item_features[
            df_item_features[ITEM_ID_COL].isin(timing_items)
        ]
        timing_item_feature_matrices[timing] = timing_datasets[
            timing
        ].build_item_features(
            (row[ITEM_ID_COL], {row["feature"]: row["weight"]})
            for index, row in timing_item_features_filtered.iterrows()
        )

all_results = []
for timing in ["Pre", "Intra", "Post"]:
    if timing in timing_models and timing in timing_train_interactions:
        # Train 평가
        train_precision = precision_at_k(
            timing_models[timing],
            timing_train_interactions[timing],
            user_features=timing_user_feature_matrices[timing],
            item_features=timing_item_feature_matrices[timing],
            k=3,
        ).mean()

        train_auc = auc_score(
            timing_models[timing],
            timing_train_interactions[timing],
            user_features=timing_user_feature_matrices[timing],
            item_features=timing_item_feature_matrices[timing],
        ).mean()

        # Test 평가
        test_precision = precision_at_k(
            timing_models[timing],
            timing_test_interactions[timing],
            user_features=timing_user_feature_matrices[timing],
            item_features=timing_item_feature_matrices[timing],
            k=3,
        ).mean()

        test_auc = auc_score(
            timing_models[timing],
            timing_test_interactions[timing],
            user_features=timing_user_feature_matrices[timing],
            item_features=timing_item_feature_matrices[timing],
        ).mean()

        all_results.append(
            {
                "Timing": timing,
                "Set": "Train",
                "Precision@3": f"{train_precision:.4f}",
                "AUC Score": f"{train_auc:.4f}",
            }
        )
        all_results.append(
            {
                "Timing": timing,
                "Set": "Test",
                "Precision@3": f"{test_precision:.4f}",
                "AUC Score": f"{test_auc:.4f}",
            }
        )

evaluation_results = pd.DataFrame(all_results)

try:
    print(evaluation_results.to_markdown(index=False))
except ImportError:
    print(evaluation_results.to_string(index=False))
print("=" * 80)

# precision@3: (모델이 예측한 것 중에 실제 선호하는 제품)/(모델 예측)
# => 실제 상호작용이 너무 적어서 적게 나오는 것이 당연함.
# 반면 AUC: 긍정적 상호작용(실제 선호)을 부정적 상호작용(무작위 비선호)보다 높은 순위로 예측하는 능력


             🌟 타이밍별 모델 평가 결과 🌟
| Timing   | Set   |   Precision@3 |   AUC Score |
|:---------|:------|--------------:|------------:|
| Pre      | Train |        0.4488 |      0.7914 |
| Pre      | Test  |        0.1891 |      0.7639 |
| Intra    | Train |        0.3628 |      0.8521 |
| Intra    | Test  |        0.2408 |      0.8086 |
| Post     | Train |        0.4096 |      0.8212 |
| Post     | Test  |        0.2811 |      0.8111 |


In [11]:
# ---------------------------------------------------------------------------------
# 8-2. [수정] intake_timing 컬럼 기반으로 타이밍 매핑 (df_item_raw에 적용)
# ---------------------------------------------------------------------------------
def map_intake_timing(intake_timing_value):
    """'intake_timing' 컬럼 값을 운동 타이밍 (Post/Pre/Intra) 리스트로 매핑"""
    if pd.isna(intake_timing_value):
        return ["Other"]

    timing_str = str(intake_timing_value).strip()

    # 🌟 이 제품이 포함될 수 있는 모든 타이밍 카테고리를 저장할 리스트 🌟
    timing_list = []

    # 순서에 상관없이 모든 키워드 포함 여부 확인
    if "운동 후" in timing_str:
        timing_list.append("Post")
    if "운동 전" in timing_str:
        timing_list.append("Pre")
    if "운동 중" in timing_str:
        timing_list.append("Intra")

    # 아무것도 해당되지 않으면 'Other'로 분류
    if not timing_list:
        return ["Other"]

    return timing_list


# df_item_raw에 'timing_category' 컬럼을 'intake_timing' 기준으로 생성
if "intake_timing" in df_item_raw.columns:
    df_item_raw["timing_category"] = df_item_raw["intake_timing"].apply(
        map_intake_timing
    )
    print(
        "✅ 제품 메타데이터에 'timing_category' 컬럼을 **intake_timing** 기반으로 생성 완료."
    )

    # 🌟 디버깅용: 타이밍별 제품 개수 확인 (선택 사항)
    # print("타이밍별 분류된 제품 개수:")
    # print(df_item_raw['timing_category'].value_counts())

else:
    print(
        "❌ 오류: df_item_raw에 'intake_timing' 컬럼이 없어 타이밍별 분류를 수행할 수 없습니다."
    )
    # 오류가 발생하면, 기존 category 기반 로직을 사용하도록 대체 (선택 사항)
    # df_item_raw['timing_category'] = df_item_raw['category'].apply(lambda x: ...)
    exit()  # 필수 컬럼이 없으므로 중단


# ---------------------------------------------------------------------------------
# 8-1. recommend_for_user 함수 (이전과 동일하게 유지)
# ---------------------------------------------------------------------------------
# 이 함수는 이미 'timing_category'를 포함하여 반환하도록 수정되었으므로 변경 불필요.
def recommend_for_user(
    user_id,
    model,
    dataset,
    user_features_matrix,
    item_features_matrix,
    df_item_raw,
    k=250,
):
    """특정 user_id에 대해 LightFM 모델 기반으로 상위 K개의 아이템을 추천합니다."""

    # model이 딕셔너리인 경우 전체 데이터셋 사용
    if isinstance(model, dict):
        # 타이밍별 모델이므로 전체 데이터셋 사용 불가
        return pd.DataFrame()

    user_id_map = dataset.mapping()[0]
    item_id_map = dataset.mapping()[2]
    item_id_rev_map = {v: k for k, v in item_id_map.items()}

    if user_id not in user_id_map:
        return pd.DataFrame()

    user_inner_id = user_id_map[user_id]
    n_items = dataset.mapping()[2].__len__()
    all_item_ids = np.arange(n_items)

    scores = model.predict(
        user_ids=[user_inner_id] * n_items,
        item_ids=all_item_ids,
        user_features=user_features_matrix,
        item_features=item_features_matrix,
    )

    top_k_indices = np.argsort(-scores)[:k]
    recommended_item_ids = [item_id_rev_map[i] for i in all_item_ids[top_k_indices]]

    recommendation_df = pd.DataFrame(
        {ITEM_ID_COL: recommended_item_ids, "Predicted_Score": scores[top_k_indices]}
    )

    # timing_category를 포함한 메타데이터와 병합
    item_display_cols = ["category", "product_name", "flavor", "timing_category"]
    available_cols = [col for col in item_display_cols if col in df_item_raw.columns]

    final_recommendations = recommendation_df.merge(
        df_item_raw.rename(columns={ITEM_ID_COL: ITEM_ID_COL})[
            available_cols + [ITEM_ID_COL]
        ],
        on=ITEM_ID_COL,
        how="left",
    )

    return final_recommendations


# ---------------------------------------------------------------------------------
# 8-3. 필터링된 추천 결과 가져오기 (다양성 강화)
# ---------------------------------------------------------------------------------
def get_filtered_recommendations(
    user_id,
    model,
    dataset,
    user_features_matrix,
    item_features_matrix,
    df_item_raw,
    k_total=250,
    timing=None,
    k_final=3,
):
    """
    전체 추천 결과를 받은 후, timing_category (리스트)로 필터링하여 최종 K_final 개를 반환하는 함수.
    🔥 최종 다양성 강화: 더 큰 후보 풀 + 점수 재분배
    """

    # model이 딕셔너리인 경우 (타이밍별 모델) 해당 타이밍 모델 사용
    if isinstance(model, dict) and timing and timing in model:
        # 타이밍별 모델 사용
        timing_model = model[timing]
        timing_dataset = dataset[timing]

        # 타이밍별 사용자 및 아이템 정보
        timing_user_map = timing_dataset.mapping()[0]
        timing_item_map = timing_dataset.mapping()[2]
        timing_item_rev_map = {v: k for k, v in timing_item_map.items()}

        if user_id not in timing_user_map:
            return pd.DataFrame()

        timing_user_inner_id = timing_user_map[user_id]
        n_items = len(timing_item_map)
        all_item_ids = np.arange(n_items)

        # 타이밍별 feature 행렬 구축
        # 타이밍별 사용자에 해당하는 features만 사용
        timing_user_map_keys = set(timing_user_map.keys())
        timing_user_features_filtered = df_user_features[
            df_user_features["user_id"].isin(timing_user_map_keys)
        ]
        timing_user_features = timing_dataset.build_user_features(
            (row["user_id"], {row["feature"]: row["weight"]})
            for index, row in timing_user_features_filtered.iterrows()
        )

        timing_items = df_interactions_with_timing[
            df_interactions_with_timing["timing_category"].apply(
                lambda x: timing in x if isinstance(x, list) else x == timing
            )
        ]["item_id"].unique()

        timing_item_features_filtered = df_item_features[
            df_item_features[ITEM_ID_COL].isin(timing_items)
        ]
        timing_item_features = timing_dataset.build_item_features(
            (row[ITEM_ID_COL], {row["feature"]: row["weight"]})
            for index, row in timing_item_features_filtered.iterrows()
        )

        # 예측
        scores = timing_model.predict(
            user_ids=[timing_user_inner_id] * n_items,
            item_ids=all_item_ids,
            user_features=timing_user_features,
            item_features=timing_item_features,
        )

        top_k_indices = np.argsort(-scores)[:k_final]
        recommended_item_ids = [
            timing_item_rev_map[i] for i in all_item_ids[top_k_indices]
        ]

        recommendation_df = pd.DataFrame(
            {
                ITEM_ID_COL: recommended_item_ids,
                "Predicted_Score": scores[top_k_indices],
            }
        )

        # 메타데이터 병합
        item_display_cols = [ITEM_ID_COL, "category", "product_name", "Predicted_Score"]
        final_cols = [
            col
            for col in item_display_cols
            if col in recommendation_df.columns or col in df_item_raw.columns
        ]

        if ITEM_ID_COL in recommendation_df.columns:
            final_recommendations = recommendation_df.merge(
                df_item_raw[[ITEM_ID_COL, "category", "product_name"]],
                on=ITEM_ID_COL,
                how="left",
            )
            return final_recommendations[final_cols]

        return recommendation_df[final_cols]

    all_recs = recommend_for_user(
        user_id,
        model,
        dataset,
        user_features_matrix,
        item_features_matrix,
        df_item_raw,
        k=k_total,
    )

    if all_recs.empty:
        return pd.DataFrame()

    # 🔥 타이밍 카테고리로 필터링 + 다양성 강화 (최종 전략)
    if timing and "timing_category" in all_recs.columns:
        # 1. 타이밍에 맞는 제품 필터링
        timing_filtered = all_recs[
            all_recs["timing_category"].apply(
                lambda x: timing in x if isinstance(x, list) else x == timing
            )
        ]

        # 2. 점수 기반 선형 샘플링: 상위 k_final개만 선택 (기본 동작)
        filtered_recs = timing_filtered.head(k_final)
    else:
        filtered_recs = all_recs.head(k_final)

    # 최종 출력 컬럼 정리
    output_cols = [ITEM_ID_COL, "category", "product_name", "Predicted_Score"]
    final_cols = [col for col in output_cols if col in filtered_recs.columns]

    return filtered_recs[final_cols]


# ---------------------------------------------------------------------------------
# 8-4. 타이밍별 추천 실행 및 출력 (사용자 ID를 2.0으로 고정하여 재실행)
# ---------------------------------------------------------------------------------
# 이전 이미지에서 사용된 '2.0' (혹은 해당 인덱스) 사용자를 재현합니다.
# df_user_clean.index가 문자열 '2.0'을 포함한다고 가정하고, 인덱스를 직접 지정합니다.
example_user_id = 555

if example_user_id in df_user_clean.index:
    print(f"\n\n{'='*60}")
    print(
        f"       ✅ 사용자 ID {example_user_id}에 대한 시나리오별 추천 결과 (intake_timing 기반)"
    )
    print(f"{'='*60}")

    # 1. 운동 전 추천 (Pre-Workout)
    print("\n--- 🏋️ 운동 전 (Pre-Workout) 추천 (상위 3개) ---")
    recs_pre = get_filtered_recommendations(
        user_id=example_user_id,
        model=model,
        dataset=timing_datasets,
        user_features_matrix=user_features_matrix,
        item_features_matrix=item_features_matrix,
        df_item_raw=df_item_raw,
        timing="Pre",
        k_final=3,
    )
    try:
        print(recs_pre.to_markdown(index=False))
    except ImportError:
        print(recs_pre.to_string(index=False))

    # 2. 운동 중 추천 (Intra-Workout)
    print("\n--- 💧 운동 중 (Intra-Workout) 추천 (상위 3개) ---")
    recs_intra = get_filtered_recommendations(
        user_id=example_user_id,
        model=model,
        dataset=timing_datasets,
        user_features_matrix=user_features_matrix,
        item_features_matrix=item_features_matrix,
        df_item_raw=df_item_raw,
        timing="Intra",
        k_final=3,
    )
    try:
        print(recs_intra.to_markdown(index=False))
    except ImportError:
        print(recs_intra.to_string(index=False))

    # 3. 운동 후 추천 (Post-Workout)
    print("\n--- 💪 운동 후 (Post-Workout) 추천 (상위 3개) ---")
    recs_post = get_filtered_recommendations(
        user_id=example_user_id,
        model=model,
        dataset=timing_datasets,
        user_features_matrix=user_features_matrix,
        item_features_matrix=item_features_matrix,
        df_item_raw=df_item_raw,
        timing="Post",
        k_final=3,
    )
    try:
        print(recs_post.to_markdown(index=False))
    except ImportError:
        print(recs_post.to_string(index=False))

    print("-" * 60)
else:
    print(
        f"\n유효한 사용자 ID({example_user_id})가 없어 타이밍별 추천을 생성할 수 없습니다."
    )

✅ 제품 메타데이터에 'timing_category' 컬럼을 **intake_timing** 기반으로 생성 완료.


       ✅ 사용자 ID 555에 대한 시나리오별 추천 결과 (intake_timing 기반)

--- 🏋️ 운동 전 (Pre-Workout) 추천 (상위 3개) ---
| product_id                | category     | product_name                          |   Predicted_Score |
|:--------------------------|:-------------|:--------------------------------------|------------------:|
| BSN_NOX_FRUITPUNCH        | 프리워크아웃 | 노익스플로드 후르츠펀치               |          -6.0262  |
| ANIMAL_PRIMAL_FRUITSPUNCH | 프리워크아웃 | 애니멀 프라이멀 프리워크아웃 과일펀치 |          -6.53514 |
| BSN_NOX_GRAPE             | 프리워크아웃 | 노익스플로드 포도                     |          -6.82147 |

--- 💧 운동 중 (Intra-Workout) 추천 (상위 3개) ---


--- 💪 운동 후 (Post-Workout) 추천 (상위 3개) ---
| product_id                  | category   | product_name                                 |   Predicted_Score |
|:----------------------------|:-----------|:---------------------------------------------|------------------:|
| OPTIMUM_GSWHEY_CHOCOLATE    | 프로틴     | 옵티멈 뉴트리션 골드스탠다

In [12]:
# ================================================================
# 🔍 타이밍별 제품 분포 확인 (디버깅)
# ================================================================
print("=" * 80)
print("🔍 타이밍별 제품 분포 디버깅")
print("=" * 80)

# 1. 전체 제품의 타이밍 분포 확인
if "timing_category" in df_item_raw.columns:
    print("\n📊 전체 제품의 timing_category 분포:")

    # timing_category가 리스트인지 확인
    def count_timing_distribution(df):
        pre_count = 0
        intra_count = 0
        post_count = 0
        both_pre_intra = 0
        both_pre_post = 0
        both_intra_post = 0
        all_three = 0

        for cat in df["timing_category"]:
            if isinstance(cat, list):
                if "Pre" in cat:
                    pre_count += 1
                if "Intra" in cat:
                    intra_count += 1
                if "Post" in cat:
                    post_count += 1

                if (
                    "Pre" in cat
                    and "Intra" in cat
                    and len([x for x in cat if x in ["Pre", "Intra", "Post"]]) == 2
                ):
                    both_pre_intra += 1
                if (
                    "Pre" in cat
                    and "Post" in cat
                    and len([x for x in cat if x in ["Pre", "Intra", "Post"]]) == 2
                ):
                    both_pre_post += 1
                if (
                    "Intra" in cat
                    and "Post" in cat
                    and len([x for x in cat if x in ["Pre", "Intra", "Post"]]) == 2
                ):
                    both_intra_post += 1
                if "Pre" in cat and "Intra" in cat and "Post" in cat:
                    all_three += 1
            else:
                if cat == "Pre":
                    pre_count += 1
                elif cat == "Intra":
                    intra_count += 1
                elif cat == "Post":
                    post_count += 1

        return {
            "Pre": pre_count,
            "Intra": intra_count,
            "Post": post_count,
            "Pre+Intra": both_pre_intra,
            "Pre+Post": both_pre_post,
            "Intra+Post": both_intra_post,
            "All three": all_three,
        }

    timing_dist = count_timing_distribution(df_item_raw)
    for key, val in timing_dist.items():
        print(f"  {key:15s}: {val:3d}개 제품")

    print(f"\n📦 총 제품 수: {len(df_item_raw)}")

    # 2. 샘플 timing_category 값 확인
    print("\n📝 timing_category 샘플 (처음 10개):")
    for idx, row in df_item_raw.head(10).iterrows():
        print(f"  {row[ITEM_ID_COL][:30]:30s} -> {row['timing_category']}")

    # 3. 각 타이밍별로 실제 사용 가능한 제품 ID 확인
    print("\n🎯 각 타이밍별 사용 가능한 제품 ID 예시 (처음 5개씩):")
    for timing in ["Pre", "Intra", "Post"]:
        matching_products = df_item_raw[
            df_item_raw["timing_category"].apply(
                lambda x: timing in x if isinstance(x, list) else x == timing
            )
        ]
        print(f"\n  {timing} ({len(matching_products)}개 제품):")
        for prod_id in matching_products[ITEM_ID_COL].head(5):
            print(f"    - {prod_id}")

else:
    print("❌ timing_category 컬럼이 없습니다!")

print("\n" + "=" * 80)

🔍 타이밍별 제품 분포 디버깅

📊 전체 제품의 timing_category 분포:
  Pre            :  86개 제품
  Intra          :  42개 제품
  Post           : 120개 제품
  Pre+Intra      :   0개 제품
  Pre+Post       :  15개 제품
  Intra+Post     :   0개 제품
  All three      :   0개 제품

📦 총 제품 수: 234

📝 timing_category 샘플 (처음 10개):
  ANIMAL_MEAL_CHOCOLATE          -> ['Post']
  CBUM_MASSGAINER_CHOCOLATE      -> ['Post']
  CBUM_MASSGAINER_COOKIESCREAM   -> ['Post']
  CBUM_MASSGAINER_VANILLA        -> ['Post']
  GASPARI_REALMASS_CHOCOLATEMILK -> ['Post']
  LABRADA_MUSCLEMASS_CHOCOLATE   -> ['Post']
  MHP_UPYOURMASS_CHOCOLATE       -> ['Post']
  REDCON_MRE_FUDGEBROWNIE        -> ['Post']
  REDCON_MRE_OATMEALCHOCOLATECHI -> ['Post']
  REDCON_MRE_PEANUTBUTTERCOOCKIE -> ['Post']

🎯 각 타이밍별 사용 가능한 제품 ID 예시 (처음 5개씩):

  Pre (86개 제품):
    - SAMDAE_INSEASON_CHERRYLEMON
    - SAMDAE_INSEASON_PLAIN
    - SAMDAE_OFFSEASON_GRAIN
    - SAMDAE_OFFSEASON_SWEETPOTATO
    - SAMDAE_PRE_GRAPE

  Intra (42개 제품):
    - ALLRIGHT_SIGNAL_SOURAPPLE
    - ANIMAL_J

In [ ]:
# ================================================================
# 10. Train/Test 사용자별 Top-3 추천 결과 생성 (Pre/Intra/Post, 부원료)
# ================================================================
import os


# 유틸: 단일 유저에 대해 시나리오별 Top-3 행들을 생성
def build_user_top10_rows(
    user_id,
    user_set_name,
    model,
    dataset,
    user_features_matrix,
    item_features_matrix,
    df_item_raw,
):
    rows = []

    # 1) Pre/Intra/Post
    for scenario in ["Pre", "Intra", "Post"]:
        recs = get_filtered_recommendations(
            user_id=user_id,
            model=model,
            dataset=dataset,
            user_features_matrix=user_features_matrix,
            item_features_matrix=item_features_matrix,
            df_item_raw=df_item_raw,
            timing=scenario,
            k_total=250,
            k_final=10,
        )
        if not recs.empty:
            for rank_idx, (_, r) in enumerate(recs.head(10).iterrows(), start=1):
                rows.append(
                    {
                        "user_set": user_set_name,
                        "user_id": user_id,
                        "scenario": scenario,
                        "rank": rank_idx,
                        "product_id": r.get(ITEM_ID_COL),
                        "category": r.get("category"),
                        "product_name": r.get("product_name"),
                        "predicted_score": r.get("Predicted_Score"),
                    }
                )

    # 2) 부원료만 별도로 Top-3 (타이밍 무관)
    all_recs = recommend_for_user(
        user_id,
        model,
        dataset,
        user_features_matrix,
        item_features_matrix,
        df_item_raw,
        k=250,
    )
    if not all_recs.empty and "ingredient_type" in df_item_raw.columns:
        # 메타데이터와 결합해서 ingredient_type 접근
        merged = all_recs.merge(
            df_item_raw[[ITEM_ID_COL, "ingredient_type", "category", "product_name"]],
            on=ITEM_ID_COL,
            how="left",
        )
        sub_ing = merged[merged["ingredient_type"] == "부원료"].head(3)
        if not sub_ing.empty:
            for rank_idx, (_, r) in enumerate(sub_ing.iterrows(), start=1):
                rows.append(
                    {
                        "user_set": user_set_name,
                        "user_id": user_id,
                        "scenario": "부원료",
                        "rank": rank_idx,
                        "product_id": r.get(ITEM_ID_COL),
                        "category": r.get("category"),
                        "product_name": r.get("product_name"),
                        "predicted_score": r.get("Predicted_Score"),
                    }
                )

    return rows


# # Train/Test 사용자 ID 목록 수집
# train_users = (
#     sorted(df_train_interactions["user_id"].unique())
#     if "df_train_interactions" in globals()
#     else []
# )
test_users = (
    sorted(df_test_interactions["user_id"].unique())
    if "df_test_interactions" in globals()
    else []
)

# print(f"Train users: {len(train_users)} | Test users: {len(test_users)}")

# # 배치 생성
# train_rows = []
# for uid in train_users:
#     train_rows.extend(
#         build_user_top10_rows(
#             uid,
#             "train",
#             model,
#             timing_datasets,
#             user_features_matrix,
#             item_features_matrix,
#             df_item_raw,
#         )
#     )

test_rows = []
from tqdm import tqdm

for uid in tqdm(test_users, desc="Building test user top10 rows"):
    test_rows.extend(
        build_user_top10_rows(
            uid,
            "test",
            model,
            timing_datasets,
            user_features_matrix,
            item_features_matrix,
            df_item_raw,
        )
    )

# 데이터프레임으로 정리
# train_top10_df = pd.DataFrame(
#     train_rows,
#     columns=[
#         "user_set",
#         "user_id",
#         "scenario",
#         "rank",
#         "product_id",
#         "category",
#         "product_name",
#         "predicted_score",
#     ],
# )
test_top10_df = pd.DataFrame(
    test_rows,
    columns=[
        "user_set",
        "user_id",
        "scenario",
        "rank",
        "product_id",
        "category",
        "product_name",
        "predicted_score",
    ],
)

# 저장 경로
out_dir = os.path.dirname(os.path.abspath("recommendations_top10_train.csv"))
# train_path = os.path.join(out_dir, "recommendations_top10_train.csv")
test_path = os.path.join(out_dir, "recommendations_top10_test.csv")

# train_top10_df.to_csv(train_path, index=False)
test_top10_df.to_csv(test_path, index=False)

print("\n[Saved]")
# print(f"- Train Top-3: {train_path}")
print(f"- Test  Top-3: {test_path}")

Building test user top10 rows:   0%|          | 0/552 [00:00<?, ?it/s]

Building test user top10 rows:  13%|█▎        | 71/552 [00:45<04:56,  1.62it/s]

In [ ]:
# ================================================================
# 다양성 분석: 추천 결과의 다양성 확인 (순위 정보 포함)
# ================================================================
print("=" * 80)
print("📊 추천 결과 다양성 분석 (순위 정보 포함)")
print("=" * 80)

# 시나리오별 제품별 추천 빈도
for scenario in ["Pre", "Intra", "Post"]:
    if scenario not in test_top10_df["scenario"].values:
        continue

    scenario_df = test_top10_df[test_top10_df["scenario"] == scenario]
    product_counts = scenario_df["product_id"].value_counts()

    print(f"\n--- {scenario} 추천 ({len(scenario_df)}건) ---")
    print(f"고유 제품 수: {len(product_counts)}개")
    print(f"상위 5개 제품:")
    for i, (product, count) in enumerate(product_counts.head(5).items(), 1):
        percentage = (count / len(scenario_df)) * 100
        product_name = (
            scenario_df[scenario_df["product_id"] == product]["product_name"].iloc[0]
            if len(scenario_df[scenario_df["product_id"] == product]) > 0
            else "N/A"
        )

        # 순위별 추천 횟수 계산
        rank_counts = (
            scenario_df[scenario_df["product_id"] == product]["rank"]
            .value_counts()
            .sort_index()
        )
        rank_str = ", ".join([f"{rank}위:{cnt}회" for rank, cnt in rank_counts.items()])

        print(f"  {i}. {product[:30]:30s} {count:4d}회 ({percentage:5.2f}%)")
        print(f"      순위 분포: {rank_str}")
        print(f"      제품명: {product_name[:40]}")

    # 다양성 지표
    unique_products = len(product_counts)
    total_recommendations = len(scenario_df)
    diversity = (unique_products / total_recommendations) * 100
    print(
        f"\n  다양성 지표: {unique_products}/{total_recommendations} = {diversity:.2f}%"
    )

    # 1위로 추천된 횟수 기준 집중도
    rank1_df = scenario_df[scenario_df["rank"] == 1]
    rank1_counts = rank1_df["product_id"].value_counts()
    rank1_percentage = (
        (rank1_counts.iloc[0] / len(rank1_df)) * 100 if len(rank1_counts) > 0 else 0
    )
    print(f"  1위 추천 집중도: {rank1_counts.iloc[0]}회 ({rank1_percentage:.2f}%)")

    # 상위 3개 제품이 차지하는 비율
    top3_percentage = (product_counts.head(3).sum() / total_recommendations) * 100
    print(f"  상위 3개 제품의 총 점유율: {top3_percentage:.2f}%")

print("\n" + "=" * 80)
print("✅ 노트북 재실행 시 다양성 개선 효과를 확인할 수 있습니다.")
print("=" * 80)

📊 추천 결과 다양성 분석 (순위 정보 포함)


NameError: name 'test_top10_df' is not defined

In [ ]:
# ================================================================
# 🔍 상호작용 데이터 제품 빈도 확인
# ================================================================
print("=" * 80)
print("🔍 상호작용 데이터 제품 빈도 분석")
print("=" * 80)

if "df_interactions" in globals():
    # 전체 상호작용 데이터 제품 빈도
    print("\n📊 전체 상호작용 데이터 제품 빈도 (상위 20개):")
    item_counts_all = df_interactions["item_id"].value_counts()
    for i, (item_id, count) in enumerate(item_counts_all.head(20).items(), 1):
        percentage = (count / len(df_interactions)) * 100
        print(f"  {i:2d}. {item_id[:45]:45s} {count:4d}회 ({percentage:5.2f}%)")

    print(f"\n📦 전체 상호작용 데이터: {len(df_interactions)}건")
    print(f"📦 고유 제품 수: {len(item_counts_all)}개")
    print(f"📦 사용자 수: {df_interactions['user_id'].nunique()}명")

    # 타이밍별로 제품을 필터링하여 확인
    if "df_item_raw" in globals() and "timing_category" in df_item_raw.columns:
        print("\n" + "=" * 80)
        print("🎯 타이밍별 실제 상호작용 제품 빈도")
        print("=" * 80)

        # 타이밍별 제품 ID 가져오기
        for timing in ["Pre", "Intra", "Post"]:
            timing_items = df_item_raw[
                df_item_raw["timing_category"].apply(
                    lambda x: timing in x if isinstance(x, list) else x == timing
                )
            ][ITEM_ID_COL].unique()

            timing_interactions = df_interactions[
                df_interactions["item_id"].isin(timing_items)
            ]

            print(f"\n--- {timing} 상호작용 (총 {len(timing_interactions)}건) ---")
            timing_item_counts = timing_interactions["item_id"].value_counts()

            print(f"고유 제품 수: {len(timing_item_counts)}개")
            print(f"상위 10개 제품:")
            for i, (item_id, count) in enumerate(
                timing_item_counts.head(10).items(), 1
            ):
                percentage = (
                    (count / len(timing_interactions)) * 100
                    if len(timing_interactions) > 0
                    else 0
                )
                print(f"  {i:2d}. {item_id[:45]:45s} {count:4d}회 ({percentage:5.2f}%)")

            # 집중도 지표
            if len(timing_item_counts) > 0:
                top1_percentage = (
                    (timing_item_counts.iloc[0] / len(timing_interactions)) * 100
                    if len(timing_interactions) > 0
                    else 0
                )
                top10_percentage = (
                    (timing_item_counts.head(3).sum() / len(timing_interactions)) * 100
                    if len(timing_interactions) > 0
                    else 0
                )
                print(f"\n  상위 1개 점유율: {top1_percentage:.2f}%")
                print(f"  상위 3개 점유율: {top10_percentage:.2f}%")
else:
    print("❌ df_interactions가 없습니다!")

print("\n" + "=" * 80)

🔍 상호작용 데이터 제품 빈도 분석
❌ df_interactions가 없습니다!

